<a href="https://colab.research.google.com/github/crescendonow/thai_geoparsing/blob/main/toponym_colab/1_1BERT_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==1.4.0. 

In [ ]:
# Install thai2transformer 
!pip -q install torch==1.5.0 torchtext==0.4.0 torchvision==0.6.0
!pip -q install transformers==3.5.0 thai2transformers==0.1.2

In [ ]:
!pip install datasets 
!pip install seqeval 
!pip install sentencepiece
!pip install pickle-mixin

In [ ]:
!pip install torch-optimizer

In [ ]:
!apt install git-lfs

In [ ]:
#Utilities lib for use
import numpy as np
import pickle
import tqdm
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification,\
TrainingArguments, Trainer, DataCollatorForTokenClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load train, validate and test token by attacut and BERT format
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/1.Tokennization/train_bert.data', 'rb') as token1:
    train_data = pickle.load(token1)
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/1.Tokennization/eval_bert.data', 'rb') as token2:
    eval_data = pickle.load(token2)
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/1.Tokennization/datasets_test.data', 'rb') as token3:
    test_data = pickle.load(token3)    

In [ ]:
#load dictionary index and processing data label_list
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/5.dict/ner_ix.dict', 'rb') as dict_ner:
    ner_ix = pickle.load(dict_ner)
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/6.Processing_variables/ner_list.data', 'rb') as list_ner:
    label_list = pickle.load(list_ner)

In [ ]:
#prepared index to BOI tags, convert index to ner
ix_to_ner = dict((v,k) for k,v in ner_ix.items()) 

In [ ]:
#list of model for use
task = "ner"
public_models = ['xlm-roberta-base', 'bert-base-multilingual-cased'] 
user_name = 'airesearch'
model_name = "wangchanberta-base-att-spm-uncased" 
batch_size = 16

In [ ]:
#define tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'{user_name}/{model_name}' if model_name not in public_models else f'{model_name}',
                revision='main',
                model_max_length=416,)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

In [ ]:
#create example for use tokenize and input ids
example = train_data[1670]
tokenized_input = tokenizer(example['tokens'], is_split_into_words=True)
tokenized_input
print(len(tokenized_input['input_ids']), len(example['ner_tags']))#create example for use tokenize and input ids

63 40


In [ ]:
print(tokenized_input)

{'input_ids': [5, 10, 32, 10, 4996, 10, 161, 10, 93, 10, 1087, 10, 1203, 10, 1432, 10, 26, 10, 93, 10, 1087, 10, 16535, 10, 82, 1241, 10, 1697, 6172, 10, 17572, 10, 1267, 10, 10351, 256, 10, 18115, 160, 10, 7406, 10, 18679, 10, 2795, 1241, 2246, 10, 9910, 10, 4178, 10, 29, 10, 451, 10, 3, 5948, 10, 3, 875, 302, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
#from upper cell rcognize length of word_ids and tags not equal so we need to algn new label 
aligned_labels = [-100 if i is None else example[f'{task}_tags'][i] for i in tokenized_input.word_ids()]
print(len(tokenized_input['input_ids']), len(aligned_labels))
aligned_labels
tokenized_input

In [ ]:
#create function for tokenize and reformat
def tokenize_and_align_labels(data):
    tokenized_datasets = []
    for rec in data:
        tokenized_input = tokenizer(rec['tokens'], is_split_into_words=True)
        aligned_labels = [-100 if i is None else rec[f'{task}_tags'][i] for i in tokenized_input.word_ids()]
        rec['input_ids'] = tokenized_input['input_ids']
        rec['attention_mask'] = tokenized_input['attention_mask']
        rec['labels'] = aligned_labels
        tokenized_datasets.append(rec)
    return tokenized_datasets

In [ ]:
train_bert = tokenize_and_align_labels(train_data)
eval_bert = tokenize_and_align_labels(eval_data)
test_bert = tokenize_and_align_labels(test_data)

In [ ]:
#prepare fine tuning model
model = AutoModelForTokenClassification.from_pretrained(f'{user_name}/{model_name}', num_labels=len(label_list))

Downloading:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier

In [ ]:
#training arguments (parameters)
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=12,
    weight_decay=0.01,
)

In [ ]:
#function for convert dataset to batch tensors
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
#computation metrics , we use seqeval for sequence tagging task
metric = load_metric("seqeval")

In [ ]:
#for seqeval not allowed format '-' so replace it by '_', pong tai i here tam goo sia wela eak, sud!!
labels = [label_list[i].replace('_', '-') for i in example[f'{task}_tags']]
metric.compute(predictions=[labels], references=[labels])

In [ ]:
#create function for computation metric
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
#fine tuning part by function Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=train_bert,
    eval_dataset=eval_bert,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
%%time
#call method train
trainer.train()

c:\Users\sveng\anaconda3\envs\geo_bert\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17955
  Num Epochs = 12
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 13476


  0%|          | 0/13476 [00:00<?, ?it/s]

The following columns in the training set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: id, pos_tags, ner_tags, tokens. If id, pos_tags, ner_tags, tokens are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to test-ner\checkpoint-500
Configuration saved in test-ner\checkpoint-500\config.json


{'loss': 0.2208, 'learning_rate': 1.925794004155536e-05, 'epoch': 0.45}


Model weights saved in test-ner\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-ner\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-ner\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test-ner\checkpoint-1000
Configuration saved in test-ner\checkpoint-1000\config.json


...

Model weights saved in test-ner\checkpoint-13000\pytorch_model.bin
tokenizer config file saved in test-ner\checkpoint-13000\tokenizer_config.json
Special tokens file saved in test-ner\checkpoint-13000\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4489
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: id, pos_tags, ner_tags, tokens. If id, pos_tags, ner_tags, tokens are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.


  0%|          | 0/281 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.11523691564798355, 'eval_precision': 0.6101921024546425, 'eval_recall': 0.6790380047505938, 'eval_f1': 0.6427768409218663, 'eval_accuracy': 0.9796842470506593, 'eval_runtime': 230.7244, 'eval_samples_per_second': 19.456, 'eval_steps_per_second': 1.218, 'epoch': 12.0}
{'train_runtime': 38745.1394, 'train_samples_per_second': 5.561, 'train_steps_per_second': 0.348, 'train_loss': 0.04289283876993706, 'epoch': 12.0}
CPU times: total: 3d 13h 54min 22s
Wall time: 10h 45min 45s


TrainOutput(global_step=13476, training_loss=0.04289283876993706, metrics={'train_runtime': 38745.1394, 'train_samples_per_second': 5.561, 'train_steps_per_second': 0.348, 'train_loss': 0.04289283876993706, 'epoch': 12.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4489
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: id, pos_tags, ner_tags, tokens. If id, pos_tags, ner_tags, tokens are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.


  0%|          | 0/281 [00:00<?, ?it/s]

{'eval_loss': 0.11523691564798355,
 'eval_precision': 0.6101921024546425,
 'eval_recall': 0.6790380047505938,
 'eval_f1': 0.6427768409218663,
 'eval_accuracy': 0.9796842470506593,
 'eval_runtime': 258.8931,
 'eval_samples_per_second': 17.339,
 'eval_steps_per_second': 1.085,
 'epoch': 12.0}

In [ ]:
predictions, labels, _ = trainer.predict(test_bert)
predictions = np.argmax(predictions, axis=2)

***** Running Prediction *****
  Num examples = 22444
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: id, pos_tags, ner_tags, tokens. If id, pos_tags, ner_tags, tokens are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.


  0%|          | 0/1403 [00:00<?, ?it/s]

In [ ]:
true_predictions = [
    [label_list[p].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

In [ ]:
results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)

{'ACP': {'precision': 0.9083969465648855, 'recall': 0.9107142857142857, 'f1': 0.9095541401273886, 'number': 784}, 'ADMIN': {'precision': 0.9004464285714285, 'recall': 0.9312096029547553, 'f1': 0.9155696777122107, 'number': 4332}, 'BSN': {'precision': 0.7857142857142857, 'recall': 0.8722466960352423, 'f1': 0.8267223382045928, 'number': 454}, 'DEP': {'precision': 0.896849593495935, 'recall': 0.9378320935175345, 'f1': 0.9168831168831169, 'number': 1882}, 'FPLACE': {'precision': 0.8397185301016419, 'recall': 0.9063291139240506, 'f1': 0.8717532467532467, 'number': 1185}, 'GOV': {'precision': 0.8895348837209303, 'recall': 0.8668555240793201, 'f1': 0.8780487804878049, 'number': 353}, 'HP': {'precision': 0.89749430523918, 'recall': 0.9099307159353349, 'f1': 0.9036697247706422, 'number': 433}, 'MKT': {'precision': 0.9370629370629371, 'recall': 0.9273356401384083, 'f1': 0.9321739130434783, 'number': 289}, 'MON': {'precision': 0.75, 'recall': 0.6976744186046512, 'f1': 0.7228915662650603, 'number'

In [ ]:
#reformat classification report
def show_3digits(evalue):
    val = round(evalue, 4)
    return val

def eliminate_decimal(evalue):
    val = int(evalue)
    return val

classification_report = pd.DataFrame.from_dict(results).transpose()

classification_report['precision'] = classification_report['precision'].apply(show_3digits)
classification_report['recall'] = classification_report['recall'].apply(show_3digits)
classification_report['f1'] = classification_report['f1'].apply(show_3digits)
classification_report['number'] = classification_report['number'].apply(eliminate_decimal)

In [ ]:
print(classification_report)

                   precision  recall      f1  number
ACP                   0.9084  0.9107  0.9096     784
ADMIN                 0.9004  0.9312  0.9156    4332
BSN                   0.7857  0.8722  0.8267     454
DEP                   0.8968  0.9378  0.9169    1882
FPLACE                0.8397  0.9063  0.8718    1185
GOV                   0.8895  0.8669  0.8780     353
HP                    0.8975  0.9099  0.9037     433
MKT                   0.9371  0.9273  0.9322     289
MON                   0.7500  0.6977  0.7229      43
NAT                   0.8136  0.8818  0.8463     203
OTHER                 0.9091  0.6250  0.7407      32
RCT                   0.8878  0.9376  0.9120     785
RES                   0.8013  0.8784  0.8381     551
ROAD                  0.8929  0.9137  0.9031    2490
RP                    0.9493  0.9572  0.9532    1075
RT                    0.8629  0.9041  0.8830    2220
STORE                 0.8455  0.7686  0.8052     363
TRAN                  0.8179  0.9127  0.8627  